# Minimum edit distance :

A basic implementation of the Levenshtein Distance algorithm to compute the minimal edit distance between two strings.

In [5]:
def del_cost():
    return 1

def ins_cost():
    return 1

def sub_cost(c1, c2):
    if c1 == c2:
        return 0
    else:
        return 2

def min_edit_distance(source, target):
    # Initialize the cost matrix
    n = len(source)
    m = len(target)
    d = [[0 for _ in range(m+1)] for _ in range(n+1)]

    # Initialize the first column
    for i in range(1, n+1):
        d[i][0] = d[i-1][0] + del_cost()

    # Initialize the first row
    for i in range(1, m+1):
        d[0][i] = d[0][i-1] + ins_cost()

    # Compute the edit distance
    for i in range(1, n+1):
        for j in range(1, m+1):
            deletion = d[i-1][j] + del_cost()
            insertion = d[i][j-1] + ins_cost()
            substitution = d[i-1][j-1] + sub_cost(source[i-1], target[j-1])
            min_cost = min(deletion, insertion, substitution)
            d[i][j] = min_cost

    # Return the edit distance
    return d[n][m]

# Example usage
w1 = 'intention'
w2 = 'execution'
print('edit distance between', repr(w1), 'and', repr(w2), 'is', min_edit_distance(w1, w2))

edit distance between 'intention' and 'execution' is 8


Now we will enrich our edit distance function to also record the backtrace and use this backtrace to print an alignment between the strings.

In [6]:
def del_cost():
    return 1

def ins_cost():
    return 1

def sub_cost(c1, c2):
    if c1 == c2:
        return 0
    else:
        return 2

def min_edit_distance(source, target, do_print_chart=False):
    # Initialization of the cost matrix and backtracking
    n = len(source)
    m = len(target)
    d = [[0 for _ in range(m+1)] for _ in range(n+1)]
    backtrack = [[None for _ in range(m+1)] for _ in range(n+1)]

    # Initialize the first column
    for i in range(1, n+1):
        d[i][0] = d[i-1][0] + del_cost()
        backtrack[i][0] = 'deletion'

    # Initialize the first row
    for i in range(1, m+1):
        d[0][i] = d[0][i-1] + ins_cost()
        backtrack[i][0] = 'insertion'

    # Compute the edit distance
    for i in range(1, n+1):
        for j in range(1, m+1):
            deletion = d[i-1][j] + del_cost()
            insertion = d[i][j-1] + ins_cost()
            substitution = d[i-1][j-1] + sub_cost(source[i-1], target[j-1])
            min_cost = min(deletion, insertion, substitution)

            # Record the move corresponding to the backtracking
            if min_cost == deletion:
                backtrack[i][j] = 'deletion'
            elif min_cost == insertion:
                backtrack[i][j] = 'insertion'
            else:
                backtrack[i][j] = 'substitution'

            d[i][j] = min_cost

    # Backtracking to generate the alignment
    align_source = ''
    align_target = ''
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and (j == 0 or backtrack[i][j] == 'deletion'):
            align_source = source[i-1] + align_source
            align_target = '-' + align_target
            i -= 1
        elif j > 0 and (i == 0 or backtrack[i][j] == 'insertion'):
            align_source = '-' + align_source
            align_target = target[j-1] + align_target
            j -= 1
        else:
            align_source = source[i-1] + align_source
            align_target = target[j-1] + align_target
            i -= 1
            j -= 1

    # Return the edit distance
    return d[n][m], align_source, align_target

# Example usage
w1 = 'intention'
w2 = 'execution'
distance, align_source, align_target = min_edit_distance(w1, w2)
print('edit distance between', repr(w1), 'and', repr(w2), 'is', distance)
print('Alignment:')
print(align_source)
print(align_target)

edit distance between 'intention' and 'execution' is 8
Alignment:
inte----ntion
---execu-tion


**Adjusting Adjacency Costs for More Intelligent Edit Distance Calculation**

Now, we aim to enhance the edit distance calculation by adjusting the cost functions to better handle spelling corrections and disordered text from the web.

We will:

1. **Revise Cost Functions:** Modify the cost functions to reflect more intelligent distance measures. Specifically, we will assign a lower cost to substitutions involving adjacent keys on a QWERTY keyboard compared to non-adjacent keys.

2. **Use QWERTY Keyboard Data:** Utilize the adjacency data provided in `qwerty_graph.txt` to guide these cost adjustments, ensuring that the calculation more accurately reflects real-world typing errors.

By implementing these changes, the goal is to make the edit distance algorithm more applicable for spelling correction and more effective for handling messy text inputs.

We are going to use the ast module to import the content of the 'qwerty_graph.txt' file as a Python expression .

To download the file : https://github.com/Rich5/Keyboard-Walk-Generators

In [13]:
import ast

def del_cost():
    return 1

def ins_cost():
    return 1

def sub_cost(c1, c2, qwerty_graph):
    if c1 == c2:
        return 0
    elif c1 in qwerty_graph and c2 in qwerty_graph[c1].values():
        return 1  # Lower cost for adjacent keys on the QWERTY keyboard
    else:
        return 2  # Higher cost for non-adjacent keys

# Edit distance calculation
def min_edit_distance(source, target, qwerty_graph):

    """
    Calculates the edit distance between two strings and generates an alignment.

    Parameters
    ----------
    source : str
        The source string.
    target : str
        The target string.
    qwerty_graph : dict
        Proximity graph of keys on a QWERTY keyboard.

    Returns
    -------
    tuple
        A tuple containing the edit distance between the two strings and the alignment.
    """

    # Initialization of the cost matrix and backtracking
    n = len(source)
    m = len(target)
    D = [[0 for _ in range(m+1)] for _ in range(n+1)]
    backtrack = [[None for _ in range(m+1)] for _ in range(n+1)]

    # Initialize the first column
    for i in range(1, n+1):
        D[i][0] = D[i-1][0] + del_cost()
        backtrack[i][0] = 'up'

    # Initialize the first row
    for j in range(1, m+1):
        D[0][j] = D[0][j-1] + ins_cost()
        backtrack[0][j] = 'left'

    # Compute the edit distance and record the backtracking
    for i in range(1, n+1):
        for j in range(1, m+1):
            deletion = D[i-1][j] + del_cost()
            insertion = D[i][j-1] + ins_cost()
            substitution = D[i-1][j-1] + sub_cost(source[i-1], target[j-1], qwerty_graph)
            min_cost = min(deletion, insertion, substitution)

            # Record the move corresponding to backtracking
            if min_cost == deletion:
                backtrack[i][j] = 'up'
            elif min_cost == insertion:
                backtrack[i][j] = 'left'
            else:
                backtrack[i][j] = 'diag'

            D[i][j] = min_cost

    # Backtracking to generate the alignment
    align_source = ''
    align_target = ''
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and (j == 0 or backtrack[i][j] == 'up'):
            align_source = source[i-1] + align_source
            align_target = '-' + align_target
            i -= 1
        elif j > 0 and (i == 0 or backtrack[i][j] == 'left'):
            align_source = '-' + align_source
            align_target = target[j-1] + align_target
            j -= 1
        else:
            align_source = source[i-1] + align_source
            align_target = target[j-1] + align_target
            i -= 1
            j -= 1

    # Return the edit distance and alignment
    return D[n][m], align_source, align_target

# Example usage
w1 = 'pxecution'
w2 = 'execution'
qwerty_file = 'qwerty_graph.txt'

with open(qwerty_file, 'r') as file:
    qwerty_graph = ast.literal_eval(file.read())

distance, align_source, align_target = min_edit_distance(w1, w2, qwerty_graph)
print('The edit distance between', repr(w1), 'and', repr(w2), 'is', distance)
print('Alignment:')
print(align_source)
print(align_target)

The edit distance between 'pxecution' and 'execution' is 2
Alignment:
-pxecution
e-xecution


# Autocorrection :

**Now we will implement a basic spelling correction algorithm based on the following simple approach:**

1. **Identify the Problem:** We have a word `w` that contains a spelling error. Our goal is to find the closest valid words from a dictionary based on edit distance.

2. **Calculate Edit Distance:** For each word in the dictionary, we will compute the edit distance between `w` and the dictionary word. Edit distance measures how many operations (insertions, deletions, substitutions) are required to transform one word into another.

3. **Generate Suggestions:** We will then sort these words by their edit distance from `w` and select the top `k` closest words.

In [14]:
import heapq # We will use Heap data structure

def auto_correction(w, dictionary, k, qwerty_graph):
    """
    Generates a list of the top k closest words from the dictionary based on edit distance to the given word.

    Parameters
    ----------
    w : str
        The word to correct.
    dictionary : list of str
        List of words in the dictionary.
    k : int
        The number of top suggestions to return.
    qwerty_graph : dict
        Proximity graph of keys on a QWERTY keyboard.

    Returns
    -------
    list of str
        The top k closest suggestions to the word w.
    """

    # Dictionary to store words and their computed edit distances from the input word
    suggestions = {}

    # Compute the edit distance for each word in the dictionary
    for word in dictionary:
        distance, _, _ = min_edit_distance(w, word, qwerty_graph)
        suggestions[word] = distance

    # Use heapq to efficiently find the k smallest distances
    top_k_suggestions = heapq.nsmallest(k, suggestions, key=suggestions.get)

    return top_k_suggestions

In [15]:
w = 'intetion'
dictionary = ['intention', 'invention', 'intervention', 'attention', 'extension']
k = 3

# Load the QWERTY graph data
with open(qwerty_file, 'r') as file:
    qwerty_graph = ast.literal_eval(file.read())

# Get the top k suggestions
suggestions = auto_correction(w, dictionary, k, qwerty_graph)
print(f'Top {k} suggestions for "{w}":')
print(suggestions)

Top 3 suggestions for "intetion":
['intention', 'invention', 'intervention']


**Now we will optimize the previous algorithm by considering the following points:**

- **Correct First Letter:** People usually get the first letter of a word correct, so we can limit our search to words that start with the same letter.

- **Similar Length:** We can narrow our search to words of the same length or similar length.

- **Phonetic Matching:** We can restrict our search to words that sound similar by using a phonetic code to group words (such as Soundex).


In [24]:
!pip install fuzzywuzzy

In [31]:
import jellyfish
from fuzzywuzzy import fuzz

def have_same_soundex(word1, word2):
    return fuzz.ratio(jellyfish.soundex(word1), jellyfish.soundex(word2)) > 75

def auto_correction_optimized(w, dictionary, k, qwerty_graph):
    """
    Generates a list of the top k closest words from the dictionary based on edit distance to the given word,
    with optimizations for starting letter, length, and phonetic similarity.

    Parameters
    ----------
    w : str
        The word to correct.
    dictionary : list of str
        List of words in the dictionary.
    k : int
        The number of top suggestions to return.
    qwerty_graph : dict
        Proximity graph of keys on a QWERTY keyboard.

    Returns
    -------
    list of str
        The top k closest suggestions to the word w.
    """

    # Initialize dictionary to store words and their computed edit distances from the input word
    suggestions = {}

    # Extract the first letter
    first_letter = w[0]

    # Compute the edit distance for each word in the dictionary with optimizations
    for word in dictionary:
        # Check if the word starts with the same letter
        if word[0] != first_letter:
            continue

        # Check length similarity ( Allow some flexibility in length)
        if abs(len(word) - len(w)) > 2:
            continue

        # Compute phonetic similarity
        if not have_same_soundex(word, w):
            continue

        # Calculate the edit distance and store it
        distance, _, _ = min_edit_distance(w, word, qwerty_graph)
        suggestions[word] = distance

    top_k_suggestions = heapq.nsmallest(k, suggestions, key=suggestions.get)

    return top_k_suggestions

In [32]:
w = 'speling'
dictionary = ['spelling', 'speeling', 'spellin', 'speling', 'spling', 'spelling']

k = 3

suggestions = auto_correction_optimized(w, dictionary, k, qwerty_graph)
print('Top suggestions:', suggestions)

Top suggestions: ['speling', 'spelling', 'speeling']
